In [1]:
import os
import time
import shutil
import tensorflow as tf
import numpy as np
import pandas as pd
import cv2
import keras
import gc
from PIL import Image
%matplotlib inline
from keras.models import Sequential,Model
from tensorflow.keras.applications import ResNet50, ResNet50V2, EfficientNetB6
from keras.layers import Dense, Activation, Dropout
import keras.backend as K
from tensorflow.keras.optimizers import Adam, RMSprop, SGD
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import BatchNormalization
from keras.layers import Conv2D, MaxPool2D, AvgPool2D, Flatten, GlobalAveragePooling2D, AveragePooling2D
from keras.callbacks import ReduceLROnPlateau, EarlyStopping

In [2]:
img_size = (512,512)
# optimizer = SGD(learning_rate=5e-5, momentum=0.9)
optimizer = Adam(learning_rate=1e-4)
foldId = 0

In [3]:
train_datagen = ImageDataGenerator(rescale=1./255)
train_df = pd.read_csv("../input/diabetic-retinopathy-preprocessed-data/trainLabels.csv")

indexes = []
for i in range(len(train_df['image'])):
    if train_df['kfold'][i] == foldId:
        indexes.append(i)
train_df.drop(indexes, axis=0, inplace=True)

train_df['image'] = [str(x)+'.jpeg' for x in train_df['image']]
train_df['level'] = [str(x) for x in train_df['level']]
train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    directory="../input/diabetic-retinopathy-preprocessed-data/Data/Data/train",
    x_col="image",
    y_col='level',
    subset="training",
    batch_size=4,
    seed=42,
    shuffle=True,
    class_mode="categorical",
    target_size=img_size)

Found 25210 validated image filenames belonging to 5 classes.


In [4]:
val_datagen = ImageDataGenerator(rescale=1./255)
val_df = pd.read_csv("../input/diabetic-retinopathy-preprocessed-data/trainLabels.csv")
indexes = []
for i in range(len(val_df['image'])):
    if val_df['kfold'][i] != foldId:
        indexes.append(i)
val_df.drop(indexes, axis=0, inplace=True)

val_df['image'] = [str(x)+'.jpeg' for x in val_df['image']]
val_df['level'] = [str(x) for x in val_df['level']]
val_generator = val_datagen.flow_from_dataframe(
    dataframe=val_df,
    directory="../input/diabetic-retinopathy-preprocessed-data/Data/Data/train",
    x_col="image",
    y_col='level',
    batch_size=8,
    seed=42,
    shuffle=True,
    class_mode="categorical",
    target_size=img_size)

Found 8404 validated image filenames belonging to 5 classes.


In [5]:
test_datagen = ImageDataGenerator(rescale=1./255)
test_df = pd.read_csv("../input/diabetic-retinopathy-preprocessed-data/Data/Data/test_labels.csv")
test_df['image'] = [str(x)+'.jpeg' for x in test_df['image']]
test_df['level'] = [str(x) for x in test_df['level']]
test_generator = test_datagen.flow_from_dataframe(
    dataframe=test_df,
    directory="../input/diabetic-retinopathy-preprocessed-data/Data/Data/test",
    x_col="image",
    y_col='level',
    batch_size=1,
    seed=42,
    shuffle=False,
    class_mode="categorical",
    target_size=img_size)

Found 53574 validated image filenames belonging to 5 classes.


/opt/conda/lib/python3.7/site-packages/keras_preprocessing/image/dataframe_iterator.py:282: UserWarning: Found 2 invalid image filename(s) in x_col="image". These filename(s) will be ignored.
  .format(n_invalid, x_col)


In [6]:
efficientNet = EfficientNetB6(include_top=False, weights='imagenet', input_shape=(img_size[0], img_size[1], 3))
output = efficientNet.layers[-1].output
output = keras.layers.Flatten()(output)
efficientNet = Model(efficientNet.input, output)

model = Sequential()
model.add(efficientNet)
model.add(Dropout(0.2))
model.add(Dense(5, activation='softmax'))

model.compile(optimizer, 
            loss='categorical_crossentropy',
            metrics=['accuracy'])

checkpoint_filepath = "./trainedmodel.h5"
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint( filepath=checkpoint_filepath,
                          save_weights_only=False,
                          monitor='val_loss',
                          mode='auto',
                          save_best_only=True)

early_stopping = EarlyStopping(
                monitor="val_loss",
                patience=3,
                verbose=1,
                mode="auto",
)

model.summary()

2022-04-21 04:53:07.682161: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-21 04:53:07.758305: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-21 04:53:07.759073: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-21 04:53:07.761097: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

165249024/165234480 [==============================] - 1s 0us/step
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
model (Functional)           (None, 589824)            40960143  
_________________________________________________________________
dropout (Dropout)            (None, 589824)            0         
_________________________________________________________________
dense (Dense)                (None, 5)                 2949125   
Total params: 43,909,268
Trainable params: 43,684,829
Non-trainable params: 224,439
_________________________________________________________________


In [7]:
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=val_generator.n//val_generator.batch_size
STEP_SIZE_TEST=test_generator.n//test_generator.batch_size

In [8]:
model.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=val_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=25, 
                    callbacks=[model_checkpoint_callback, early_stopping])

/opt/conda/lib/python3.7/site-packages/keras/engine/training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '
2022-04-21 04:53:17.168374: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/25


2022-04-21 04:53:40.149077: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


6302/6302 [==============================] - 4477s 706ms/step - loss: 1.5833 - accuracy: 0.6789 - val_loss: 1.9160 - val_accuracy: 0.7590


/opt/conda/lib/python3.7/site-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


Epoch 2/25
6302/6302 [==============================] - 4443s 705ms/step - loss: 1.3162 - accuracy: 0.7178 - val_loss: 1.9835 - val_accuracy: 0.7632
Epoch 3/25
6302/6302 [==============================] - 4463s 708ms/step - loss: 1.2564 - accuracy: 0.7276 - val_loss: 1.9831 - val_accuracy: 0.7451
Epoch 4/25
6302/6302 [==============================] - 4468s 709ms/step - loss: 1.0663 - accuracy: 0.7454 - val_loss: 0.7801 - val_accuracy: 0.8202
Epoch 5/25
6302/6302 [==============================] - 4455s 707ms/step - loss: 0.8151 - accuracy: 0.7818 - val_loss: 1.2905 - val_accuracy: 0.7917
Epoch 6/25
6302/6302 [==============================] - 4456s 707ms/step - loss: 0.7001 - accuracy: 0.8037 - val_loss: 1.2929 - val_accuracy: 0.1419
Epoch 7/25
6302/6302 [==============================] - 4463s 708ms/step - loss: 0.6147 - accuracy: 0.8231 - val_loss: 1.2222 - val_accuracy: 0.6004
Epoch 00007: early stopping


In [9]:
test_generator.reset()
pred=model.predict_generator(test_generator,
                            steps=STEP_SIZE_TEST,
                            verbose=1)

/opt/conda/lib/python3.7/site-packages/keras/engine/training.py:2035: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  warnings.warn('`Model.predict_generator` is deprecated and '


53574/53574 [==============================] - 2141s 40ms/step


In [10]:
predicted_class_indices=np.argmax(pred,axis=1)
labels = (train_generator.class_indices)
labels = dict((v,k) for k,v in labels.items())

predictions = [labels[k] for k in predicted_class_indices]
filenames=test_generator.filenames

res = {filenames[i]: predictions[i] for i in range(len(filenames))}

In [11]:
from collections import Counter
P = Counter(predictions)
T = Counter(test_df['level'])
print(sorted(P.items()))
print(sorted(T.items()))

[('0', 42564), ('1', 10568), ('2', 442)]
[('0', 39533), ('1', 3762), ('2', 7861), ('3', 1214), ('4', 1206)]


In [12]:
correct = 0
predLabels = []
indexes = []
for i in range(len(test_df['image'])):
    if test_df['image'][i] in res:
        predLabels.append(res[test_df['image'][i]])
        if res[test_df['image'][i]] == test_df['level'][i]:
            correct += 1
    else:
        predLabels.append("NA")
        indexes.append(i)
acc = (correct / len(test_df['level'])) * 100
print(acc)

test_df['predLevel'] = predLabels
test_df.drop(indexes, axis=0, inplace=True)
test_df.to_csv("./predictions.csv", index=False)

model.save('model.h5')

59.59198148424668
